**It's not necessary to run this notebook**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import settings as conf
from metadata import RAPID_GWAS_PHENO_INFO

# Load UKB pheno info file

In [3]:
RAPID_GWAS_PHENO_INFO.shape

(4359, 9)

In [4]:
RAPID_GWAS_PHENO_INFO.head()

,description,variable_type,source,n_non_missing,n_missing,n_controls,n_cases,PHESANT_transformation,notes
phenotype,,,,,,,,,
G54,Diagnoses - main ICD10: G54 Nerve root and ple...,categorical,icd10,361194,0,361051.0,143.0,NaN,NaN
C21,Diagnoses - main ICD10: C21 Malignant neoplasm...,categorical,icd10,361194,0,361055.0,139.0,NaN,NaN
F45,Diagnoses - main ICD10: F45 Somatoform disorders,categorical,icd10,361194,0,360953.0,241.0,NaN,NaN
I35,Diagnoses - main ICD10: I35 Nonrheumatic aorti...,categorical,icd10,361194,0,360293.0,901.0,NaN,NaN
D38,Diagnoses - main ICD10: D38 Neoplasm of uncert...,categorical,icd10,361194,0,361087.0,107.0,NaN,NaN


# Pre-select phenotypes

In [5]:
ukb_phenos_selection = RAPID_GWAS_PHENO_INFO[
    (RAPID_GWAS_PHENO_INFO['source'].isin(('icd10', 'finngen'))) |
    ( (RAPID_GWAS_PHENO_INFO['source'] == 'phesant') & (~RAPID_GWAS_PHENO_INFO.index.str.endswith('_irnt')) )
]

In [6]:
ukb_phenos_selection.shape

(4083, 9)

# Detect duplicated phenotypes

In [11]:
from entity import Trait

In [12]:
phenos_used_counts = ukb_phenos_selection['description'].value_counts()
display(phenos_used_counts.head())

Job coding: personnel or industrial relations officer, recruitment consultant    5
Job coding: other assembler, routine operative                                   4
Job coding: assembler, machine operator, foreman                                 3
Job coding: vocational or industrial trainer/instructor                          3
Job coding: librarian, assistant librarian                                       3
Name: description, dtype: int64

In [13]:
repeated_traits = phenos_used_counts[phenos_used_counts > 1]
display(repeated_traits.shape)
repeated_traits = repeated_traits.sort_index()

(70,)

In [14]:
# repeated_traits_set = set(repeated_traits.index)

In [15]:
## Auxiliary code to detect duplicates

# the code below is used to see how to handle duplicated entries
# _tmp = set(repeated_traits.index)

# len(_tmp)

# _tmp

# phenos_used[phenos_used['description'].notna() & (phenos_used['description'] == 'Asthma')]

In [16]:
ukb_traits = [Trait(t) for t in ukb_phenos_selection.index]

The code below was not tested yet

In [ ]:
import yaml
from clustering.biclustering.analysis import Trait

len(ukb_traits)

ukb_traits[:5]

file_template = '{trait_code}'

class Object:
    pass

filename_to_description = {}
phenos_used_info_map = phenos_used.set_index('phenotype')

ukb_traits_removed = []
ukb_traits_codes_removed = []

for t in ukb_traits:
    try:
        trait = Trait(t)
    except KeyError:
        trait = Object()
        trait.trait_code = t
        trait.trait_plain_name = None
    
    trait_code = trait.trait_code
    filename = file_template.format(trait_code=trait_code)
    
    trait_plain_name = trait.trait_plain_name
    if pd.isnull(trait_plain_name):
        if trait_code in gtex_gwas_phenos_info_map.index:
            trait_plain_name = gtex_gwas_phenos_info_map.loc[trait_code, 'new_Phenotype']
        else:
            trait_plain_name = filename
    
    if trait_code in phenos_used_info_map.index:
        trait_description = phenos_used_info_map.loc[trait_code, 'description']
    elif trait_code in gtex_gwas_phenos_info_map.index:
        trait_description = gtex_gwas_phenos_info_map.loc[trait_code, 'new_Phenotype']
    else:
        raise ValueError(f'No description for pheno code: {trait_code}')
    
    if trait_description in repeated_traits_set:
        if (
             trait_code.startswith('22601_') or # Job codings
             trait_code.startswith('20003_') or # Medication vitamin c
             (trait_description in (
                 'Weight',
                 'Poultry intake',
                 'Other and unspecified dermatitis',
                 'Certain infectious and parasitic diseases',
                 'Body mass index (BMI)',
                 'Beef intake',
             ))
           ):
            trait_plain_name = trait_plain_name + f'_{trait_code}'
        elif (trait_code in ('SNORING',)):
            trait_plain_name = trait_plain_name + '_FinnGen'
        elif (trait_code.startswith('C3_')) or (trait_code in (
            'G6_NEURODEG',
            'LUNG_CANCER',
            'J10_ASTHMA_MAIN',
        )):
            ukb_traits_removed.append(t)
            ukb_traits_codes_removed.append(trait_code)
            continue
    
    filename_to_description[filename] = trait_plain_name

# should be empty
[(k, v) for k, v in filename_to_description.items() if pd.isnull(v)]

assert len(ukb_traits_removed) == len(ukb_traits_codes_removed)

len(ukb_traits_removed)

len(filename_to_description)

assert len(ukb_traits_removed) + len(filename_to_description) == 4091

assert len(filename_to_description) == len(set(filename_to_description.values())), len(set(filename_to_description.values()))

with open('/mnt/tmp/ukb_filename_maps.yml', 'w') as outfile:
    yaml.dump({'ukb_name': filename_to_description}, outfile, default_flow_style=False, default_style='yaml_dump')

with open('/mnt/tmp/ukb_trait_codes_removed.txt', 'w') as f:
    for tc in ukb_traits_codes_removed:
        f.write(f'{tc}\n')